# "No" of the mental Health"

1st area:

In [ ]:
from google.colab import drive, userdata
import google.generativeai as genai
import os
import pandas as pd
from tqdm import tqdm
import time
import random
from google.api_core.exceptions import ResourceExhausted

# Securely configure Gemini API Key using Google Colab Secrets
try:
    genai.configure(api_key=userdata.get("GOOGLE_API_KEY_2"))
except Exception as e:
    print(f"Error configuring Gemini API: {e}")
    print("Please ensure you have set 'GEMINI_API_KEY' in Google Colab Secrets (key icon on the left sidebar).")
    exit()

# Initialize the Generative Model with Gemini 1.5 Flash.
model = genai.GenerativeModel('gemini-2.0-flash') # Using gemini-2.0-flash as it's powerful and cost-effective

# Mount Google Drive
drive.mount('/content/drive')

# --- Categories for Non-Mental Health Content ---
non_mh_categories = {
    'Technology & Gadgets': [
        "latest smartphone review", "future of AI in daily life", "smart home automation tips",
        "gaming console comparison", "cybersecurity best practices", "wearable tech innovations",
        "how to build a custom PC", "review of a new software update", "evolution of virtual reality"
    ],
    'Travel & Tourism': [
        "hiking trails in Patagonia", "top 10 beaches in Thailand", "exploring ancient Roman ruins",
        "budget travel tips for Europe", "culinary tour of Italy", "wildlife safari in Africa",
        "backpacking essentials", "guide to national parks in North America", "visiting historical landmarks"
    ],
    'Cooking & Recipes': [
        "easy weeknight dinner recipes", "mastering sourdough bread", "vegan meal prep ideas",
        "history of Italian pasta", "best barbecue techniques", "baking perfect cookies",
        "fermentation basics", "review of a new kitchen gadget", "seasonal vegetable dishes"
    ],
    'Home Improvement & DIY': [
        "renovating a bathroom on a budget", "gardening for beginners", "building a backyard deck",
        "interior design trends", "DIY furniture projects", "painting tips for a smooth finish",
        "smart home integration", "creating a sustainable garden", "repairing leaky faucets"
    ],
    'Sports & Fitness': [
        "training for a marathon", "NBA season preview", "yoga for flexibility",
        "benefits of strength training", "history of the Olympic Games", "soccer match analysis",
        "cycling routes for adventurers", "nutrition for athletes", "review of new running shoes"
    ],
    'Finance & Economics': [
        "personal budgeting strategies", "stock market trends for Q3", "understanding cryptocurrency",
        "real estate investment tips", "retirement planning basics", "global economic outlook",
        "starting a small business", "managing student debt", "impact of interest rates on loans"
    ],
    'Science & Nature (non-medical)': [
        "discoveries in astrophysics", "the life cycle of butterflies", "climate change effects on oceans",
        "geology of volcanoes", "robotics in manufacturing", "new findings in quantum physics",
        "observing constellations", "marine biology research", "sustainable energy solutions"
    ],
    'History & Culture': [
        "ancient Egyptian pharaohs", "the Renaissance art movement", "traditions of Japanese tea ceremony",
        "World War II pivotal moments", "famous literary figures", "evolution of music genres",
        "exploring indigenous cultures", "architecture of medieval castles", "cultural festivals worldwide"
    ],
    'Everyday Life (neutral)': [
        "tips for organizing your workspace", "how to choose the right pet", "planning a successful road trip",
        "daily commute challenges", "best ways to relax after work (non-mental health)", "volunteering in your community",
        "a day in the life of a city dweller", "the benefits of reading fiction", "understanding local zoning laws"
    ]
}

# Specific prompts for "confusing" keywords in non-MH contexts
confusing_prompts = [
    {"topic": "Travel", "prompt": "Write a short travel blog post about an 'emotional trip' through a historical city, focusing on the historical impact and personal awe, not psychological states. The emotions should be awe, wonder, fascination, not sadness or anxiety."},
    {"topic": "Engineering", "prompt": "Describe a 'stress test' performed on a new bridge design. Focus on the structural integrity, engineering principles, and physical limits, not human stress."},
    {"topic": "Finance", "prompt": "Discuss a 'depressed economy' and its indicators. Explain economic terms like recession, inflation, and unemployment, without focusing on the psychological impact on individuals."},
    {"topic": "Sports", "prompt": "Detail the 'anxious wait' for the final results of a major sporting event. Describe the tension of the game, the scores, and the anticipation of fans, not clinical anxiety."},
    {"topic": "Botany", "prompt": "Write about the 'healing properties' of a specific herb. Focus on its chemical compounds, traditional uses as a physical remedy, and cultivation, not mental or emotional healing."},
    {"topic": "DIY", "prompt": "Explain how to fix a 'broken window' at its 'breaking point'. Focus on the physical repair process, tools, and materials needed."},
    {"topic": "Daily Life", "prompt": "Write a short piece advising people to 'mind the gap' on a subway platform. Explain it as a safety instruction, not a mental state."},
    {"topic": "Culinary", "prompt": "Describe how to create a 'depressed' or 'sunken' cake, explaining the baking process and common baking errors."},
    {"topic": "Weather", "prompt": "Describe a 'blue' sky on a clear day, or a 'down' pour of rain. Focus on meteorological descriptions."},
    {"topic": "Music", "prompt": "Discuss the 'mood' of a particular piece of classical music, describing its tempo, instrumentation, and emotional impact on the listener purely from an aesthetic perspective, not mental health states."}
]


def generate_non_mental_health_entry(category, sub_topic, entry_number, is_confusing_keyword=False):
    """
    Generates a single non-mental health entry based on category and sub-topic.
    Includes retry mechanism and length enforcement.
    `is_confusing_keyword` flag modifies prompts to use keywords that might be
    mistaken for mental health topics but are used in a non-MH context.
    """
    max_retries = 3
    base_length_min = 500
    base_length_max = 2000 # Keep content concise but informative

    for attempt in range(max_retries):
        if is_confusing_keyword:
            # For confusing keywords, directly use the specific prompt
            prompt_detail = sub_topic # Here sub_topic is actually the full prompt from confusing_prompts
            final_prompt = f"""
            Generate a concise, factual, and engaging piece of text NOT related to mental health.
            Focus on the following specific scenario:
            {prompt_detail}

            Ensure the content clearly belongs to a non-mental health domain (e.g., engineering, sports, finance, travel, botany, daily safety).
            Avoid any language that could be interpreted as describing a psychological condition, emotional struggle (beyond superficial, common-language usage like 'anticipation' or 'awe' in non-clinical contexts), therapy, diagnosis, or mental well-being.
            Structure: Title, then content.
            Length: Total content (excluding title) should be between {base_length_min} and {base_length_max} characters.
            """
        else:
            # For regular non-MH entries, use the category and sub-topic
            final_prompt = f"""
            Generate a concise, factual, and engaging article, review, or descriptive piece on the topic of "{sub_topic}" within the category of "{category}".
            The content MUST NOT be related to mental health, psychology, emotional struggles, or therapy in any way.
            Focus purely on the factual, technical, descriptive, or practical aspects of the subject.

            Example styles: blog post, news report, product review, travel guide, recipe, technical explanation, historical overview, DIY guide.

            Structure:
            1.  **Title:** A clear, informative title (20-120 characters) directly related to the topic.
            2.  **Content:** A well-structured, coherent body of text.
                - For "Travel & Tourism": Describe destinations, practical tips, historical facts, local cuisine.
                - For "Technology & Gadgets": Explain features, performance, user experience, market trends.
                - For "Cooking & Recipes": Provide instructions, ingredients, culinary tips, background.
                - For "Home Improvement & DIY": Detail steps, tools, materials, design ideas.
                - For "Sports & Fitness": Report on events, explain techniques, discuss benefits, analyze performance.
                - For "Finance & Economics": Explain concepts, analyze markets, offer practical advice, report on trends.
                - For "Science & Nature": Describe phenomena, explain theories, report on discoveries, discuss species.
                - For "History & Culture": Narrate events, describe art, explain traditions, outline biographies.
                - For "Everyday Life (neutral)": Give advice on mundane tasks, describe observations, or neutral personal experiences.

            Ensure the language is entirely devoid of mental health jargon or implications.
            Length: Total content (excluding title) should be between {base_length_min} and {base_length_max} characters.
            """
        try:
            response = model.generate_content(
                final_prompt,
                generation_config=genai.types.GenerationConfig(
                    temperature=0.7, # Slightly lower temperature for more factual content
                    max_output_tokens=base_length_max + 200, # Allow a bit extra for safety
                    stop_sequences=[]
                )
            )

            text = response.text.strip()
            first_newline_idx = text.find('\n')

            if first_newline_idx != -1:
                title = text[:first_newline_idx].strip()
                content = text[first_newline_idx+1:].strip()
            else:
                # If no newline, try to find a reasonable split point or just take first part as title
                first_period_idx = text.find('.')
                if first_period_idx != -1 and first_period_idx < 120: # Title max length check
                    title = text[:first_period_idx+1].strip()
                    content = text[first_period_idx+1:].strip()
                else:
                    # Fallback for very short or oddly formatted responses
                    title = text[:80].strip() + "..." if len(text) > 80 else text.strip()
                    content = text[80:].strip() if len(text) > 80 else ""

            if len(title) > 120:
                title = title[:117] + "..."

            # Basic content length check
            if len(content) < base_length_min:
                print(f"Warning: Content for non-MH entry {category}/{sub_topic} ({'confusing' if is_confusing_keyword else 'normal'}) entry {entry_number} too short ({len(content)} chars). Retrying...")
                time.sleep(2)
                continue

            return {
                'category': category,
                'sub_topic': sub_topic,
                'title': title,
                'content': content,
                'is_mental_health': False, # This is the crucial label
                'is_confusing_keyword_type': is_confusing_keyword # To track confusing entries
            }

        except ResourceExhausted as e:
            print(f"Error generating non-MH content for {category}/{sub_topic} ({'confusing' if is_confusing_keyword else 'normal'}) entry {entry_number} (Attempt {attempt+1}/{max_retries}): Quota Exceeded. Skipping further retries for this specific entry due to persistent quota error.")
            return {
                'category': category,
                'sub_topic': sub_topic,
                'title': f"Failed (Quota) for {sub_topic}",
                'content': "Content generation failed due to API quota limits.",
                'is_mental_health': False,
                'is_confusing_keyword_type': is_confusing_keyword
            }
        except Exception as e:
            print(f"Error generating non-MH content for {category}/{sub_topic} ({'confusing' if is_confusing_keyword else 'normal'}) entry {entry_number} (Attempt {attempt+1}/{max_retries}): {e}")
            time.sleep(5)
            continue

    print(f"Failed to generate non-MH content for {category}/{sub_topic} ({'confusing' if is_confusing_keyword else 'normal'}) entry {entry_number} after {max_retries} attempts.")
    return {
        'category': category,
        'sub_topic': sub_topic,
        'title': f"Failed to generate for {sub_topic}",
        'content': "Content generation failed.",
        'is_mental_health': False,
        'is_confusing_keyword_type': is_confusing_keyword
    }

# --- Generation Logic for Non-Mental Health Data ---
output_dir = '/content/drive/MyDrive/Mental Health'
# Save to a new CSV file for non-mental health data
non_mh_output_path = os.path.join(output_dir, 'non_mental_health_dataset.csv')
os.makedirs(output_dir, exist_ok=True)

# Load existing non-MH data if the file exists, otherwise start with an empty DataFrame
if os.path.exists(non_mh_output_path):
    print(f"Loading existing non-mental health dataset from {non_mh_output_path}...")
    df_existing_non_mh = pd.read_csv(non_mh_output_path)
    non_mh_records = df_existing_non_mh.to_dict('records')
    print(f"Loaded {len(non_mh_records)} existing non-mental health entries.")
else:
    non_mh_records = []
    print("No existing non-mental health dataset found. Starting with an empty dataset.")

# Generation parameters for new non-MH entries
# Target: ~750 new entries
# Number of main non-MH category sub_topics: 9 categories * ~9 sub-topics = ~81
# Number of confusing prompts: 10
# Let's aim for 8-9 entries per standard sub-topic and 10-15 per confusing prompt.
# We'll split the 750 target: e.g., 600 regular, 150 confusing
# Regular: 600 / (len(non_mh_categories) * avg_subtopics_per_category)
# approx 81 subtopics. 600 / 81 = ~7.4
num_regular_entries_per_subtopic = 7 # Aim for 7 entries per regular sub-topic (7 * ~81 = ~567)
num_confusing_entries_per_prompt = 15 # Aim for 15 entries per confusing prompt (15 * 10 = 150)

print(f"\nGenerating new non-mental health entries (approx. {len(non_mh_categories.keys()) * len(non_mh_categories[list(non_mh_categories.keys())[0]]) * num_regular_entries_per_subtopic + len(confusing_prompts) * num_confusing_entries_per_prompt} total)...")

# Generate regular non-MH entries
for category_name, sub_topics in non_mh_categories.items():
    for sub_topic in tqdm(sub_topics, desc=f"Generating regular non-MH '{category_name}' entries"):
        for i in range(num_regular_entries_per_subtopic):
            entry = generate_non_mental_health_entry(category_name, sub_topic, i + 1, is_confusing_keyword=False)
            non_mh_records.append(entry)
            time.sleep(0.5) # Small delay

# Generate "confusing" non-MH entries
for j, conf_item in enumerate(tqdm(confusing_prompts, desc="Generating 'confusing' non-MH entries")):
    for k in range(num_confusing_entries_per_prompt):
        entry = generate_non_mental_health_entry(conf_item['topic'], conf_item['prompt'], k + 1, is_confusing_keyword=True)
        non_mh_records.append(entry)
        time.sleep(0.5) # Small delay

# Create DataFrame from all non-MH records (existing + new) and save
df_non_mh_new = pd.DataFrame(non_mh_records)

df_non_mh_new.to_csv(non_mh_output_path, index=False)
print(f"\nTotal non-mental health entries generated and saved to {non_mh_output_path}: {len(df_non_mh_new)}")

# Display a sample of the updated non-MH data
print("\nSample of updated non-mental health data:")
print(df_non_mh_new.tail()) # Show tail to see newer entries

print("\nEntry counts by category and confusing type:")
# Use fill_value=0 for combinations that don't exist
print(df_non_mh_new.groupby(['category', 'is_confusing_keyword_type']).size().unstack(fill_value=0))
print(f"\nTotal non-mental health entries: {len(df_non_mh_new)}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
No existing non-mental health dataset found. Starting with an empty dataset.

Generating new non-mental health entries (approx. 717 total)...


Generating regular non-MH 'Technology & Gadgets' entries:   0%|          | 0/9 [00:00<?, ?it/s]

Generating regular non-MH 'Technology & Gadgets' entries:  22%|██▏       | 2/9 [00:49<02:41, 23.12s/it]WARNING:tornado.access:429 POST /v1beta/models/gemini-2.0-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 2125.68ms


Error generating non-MH content for Technology & Gadgets/smart home automation tips (normal) entry 2 (Attempt 1/3): 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.


Generating regular non-MH 'Technology & Gadgets' entries:  33%|███▎      | 3/9 [01:33<03:16, 32.68s/it]

...

Generating 'confusing' non-MH entries:  90%|█████████ | 9/10 [18:09<01:59, 119.70s/it]

Failed to generate non-MH content for Music/Discuss the 'mood' of a particular piece of classical music, describing its tempo, instrumentation, and emotional impact on the listener purely from an aesthetic perspective, not mental health states. (confusing) entry 1 after 3 attempts.
Failed to generate non-MH content for Music/Discuss the 'mood' of a particular piece of classical music, describing its tempo, instrumentation, and emotional impact on the listener purely from an aesthetic perspective, not mental health states. (confusing) entry 3 after 3 attempts.
Failed to generate non-MH content for Music/Discuss the 'mood' of a particular piece of classical music, describing its tempo, instrumentation, and emotional impact on the listener purely from an aesthetic perspective, not mental health states. (confusing) entry 4 after 3 attempts.
Failed to generate non-MH content for Music/Discuss the 'mood' of a particular piece of classical music, describing its tempo, instrumentation, and emo

Error generating non-MH content for Music/Discuss the 'mood' of a particular piece of classical music, describing its tempo, instrumentation, and emotional impact on the listener purely from an aesthetic perspective, not mental health states. (confusing) entry 11 (Attempt 1/3): 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.
Failed to generate non-MH content for Music/Discuss the 'mood' of a particular piece of classical music, describing its tempo, instrumentation, and emotional impact on the listener purely from an aesthetic perspective, not mental health states. (confusing) entry 11 after 3 attempts.
Failed to generate non-MH content for Music/Discuss the 'mood' of a particular piece of classical music, describing its tempo, inst

Generating 'confusing' non-MH entries: 100%|██████████| 10/10 [20:31<00:00, 123.12s/it]


Total non-mental health entries generated and saved to /content/drive/MyDrive/Mental Health/non_mental_health_dataset.csv: 717

Sample of updated non-mental health data:
    category                                          sub_topic  \
712    Music  Discuss the 'mood' of a particular piece of cl...   
713    Music  Discuss the 'mood' of a particular piece of cl...   
714    Music  Discuss the 'mood' of a particular piece of cl...   
715    Music  Discuss the 'mood' of a particular piece of cl...   
716    Music  Discuss the 'mood' of a particular piece of cl...   

                                                 title  \
712  Failed to generate for Discuss the 'mood' of a...   
713  Failed to generate for Discuss the 'mood' of a...   
714  Failed to generate for Discuss the 'mood' of a...   
715  ### Shostakovich's Waltz No. 2: A Dance of Sha...   
716                  ### Alpine Dawn: Smetana's Moldau   

                                               content  is_mental_health  \
7

still a lot of duplicates

In [ ]:
from google.colab import drive, userdata
import google.generativeai as genai
import os
import pandas as pd
from tqdm import tqdm
import time
import random
import hashlib # For creating content hashes
from google.api_core.exceptions import ResourceExhausted, GoogleAPIError

# --- Configuration ---
# Securely configure Gemini API Key using Google Colab Secrets
try:
    genai.configure(api_key=userdata.get("GOOGLE_API_KEY_3"))
except Exception as e:
    print(f"Error configuring Gemini API: {e}")
    print("Please ensure you have set 'GOOGLE_API_KEY_3' in Google Colab Secrets (key icon on the left sidebar).")
    exit()

# Initialize the Generative Model with Gemini 1.5 Flash.
model = genai.GenerativeModel('gemini-2.0-flash')

# Mount Google Drive
drive.mount('/content/drive')

# --- List of mental health conditions ---
diseases = [
    'acute stress disorder', 'chronic stress', 'episodic acute stress',
    'obsessive-compulsive disorder (contamination type)', 'obsessive-compulsive disorder (symmetry type)',
    'obsessive-compulsive disorder (checking type)', 'generalized anxiety disorder',
    'major depressive disorder', 'postpartum depression', 'atypical depression',
    'panic disorder with agoraphobia', 'separation anxiety disorder',
    'single event trauma (PTSD)', 'complex trauma (C-PTSD)', 'developmental trauma',
    'social anxiety disorder', 'body dysmorphic disorder', 'bipolar II disorder',
    'borderline personality disorder (emotional dysregulation focus)', 'eating disorder (anorexia nervosa)'
]

# --- Prompt Engineering for Gemini 1.5/2.0 Flash ---

def generate_entry(condition, style, entry_sequence_num, is_edge_case=False):
    """
    Generates a single mental health entry.
    Includes robust retry mechanism with exponential backoff for API errors.
    Returns the entry dictionary on success, None on failure (content too short, duplicate, or persistent API errors).
    """
    max_retries = 8 # Increased retries for better success rate
    initial_delay = 1 # seconds
    min_content_length = 1500 # Minimum length for content (excluding title)
    max_content_length = 9000 # Maximum length for content (excluding title)

    for attempt in range(max_retries):
        current_delay = initial_delay * (2 ** attempt) + random.uniform(0, 1) # Exponential backoff with jitter

        # Add a unique identifier to the prompt to encourage different responses
        # This will help Gemini know it's a new request even for the same base condition/style
        unique_prompt_seed = f"Generation attempt {entry_sequence_num}, run_id: {int(time.time()) + random.randint(0, 1000)}"

        if style == 'doctor':
            base_prompt_instructions = f"""
            You are a highly experienced, board-certified psychiatrist or clinical psychologist writing a comprehensive, empathetic, and evidence-based article for a reputable mental health awareness website, targeting a general audience. The goal is to inform, destigmatize, and offer hope.

            **Condition Focus:** {condition}
            **Tone:** Professional, empathetic, clear, authoritative yet accessible.
            **Language:** Use clear, concise language, explaining any technical terms simply.
            """
            if is_edge_case:
                edge_case_instructions = f"""
                **Special Focus (Edge Case for Doctor's Perspective):** For this specific article, focus on a **less common or particularly challenging aspect** of {condition}. Choose ONE of the following to elaborate on:
                * An **atypical symptom presentation** (e.g., masked symptoms, presentation in specific overlooked demographics like the elderly or culturally diverse groups, or symptoms that strongly mimic physical illness).
                * **Complex comorbidities** that significantly complicate diagnosis, prognosis, or treatment planning for {condition} (e.g., co-occurring severe physical illness, multiple personality disorders, or complex neurological conditions).
                * Challenges in **differential diagnosis** when {condition} presents similarly to other, vastly different conditions, requiring nuanced clinical judgment.
                * **Resistance to standard treatments** or the necessity for highly specialized, multidisciplinary, or novel interventions (e.g., intensive inpatient care, specific neuro-modulation techniques, very integrated care models) not commonly discussed.
                * Profound **cultural or socio-economic factors** that uniquely influence the presentation, stigma, or access to care for {condition} in specific populations.
                * A focus on a specific, **less-discussed subtype or manifestation** of {condition} that poses unique clinical challenges.
                * **Your goal is to provide deep, insightful clinical perspective on this specific challenging facet.** Avoid generic information; make this article distinct from typical overviews.
                """
            else:
                edge_case_instructions = f"""
                **Special Focus (Typical Case for Doctor's Perspective):** For this specific article, provide a deep dive into the **typical and most common presentations, diagnostic pathways, and first-line, evidence-based treatment strategies** for {condition}. Choose ONE of the following to elaborate on:
                * The **classic symptomology** and how it commonly impacts an individual's functioning across various life domains.
                * The **standard diagnostic process**, including common assessment tools and criteria used in routine clinical practice.
                * The **most widely accepted and effective first-line treatments**, detailing how they work and what a typical treatment journey involves.
                * **Early intervention strategies** for {condition} and their impact on long-term prognosis.
                * **Psychoeducation** for patients and families about managing {condition} in a typical course.
                * **Your goal is to provide a detailed, yet accessible, overview of the common clinical picture and standard management.** Avoid highly niche or unusual scenarios.
                """

            prompt = f"""
            {base_prompt_instructions}
            {edge_case_instructions}

            **Required Structure & Content (Strictly Adhere to Lengths and Sections):**
            1.  **Title:** A compelling, informative title (10-100 characters). Should clearly indicate the condition and, if applicable, hint at the specific focus (e.g., "Understanding Atypical Presentations of PTSD").
            2.  **Introduction (200-500 characters):** Briefly define the condition and its prevalence. Hook the reader with a relevant statistic or a compelling statement related to the article's specific focus.
            3.  **Clinical Overview & Diagnostic Nuances (700-2000 characters):**
                * Detailed definition and specific diagnostic criteria relevant to your chosen focus (typical or edge case).
                * Common and, crucially, *atypical or nuanced* symptoms (emotional, physical, cognitive, behavioral) as they relate to the article's specific angle.
                * Potential triggers and risk factors, highlighting any less obvious ones relevant to the specific focus.
                * Brief explanation of underlying mechanisms or contributing factors.
            4.  **Impact on Daily Life (300-800 characters):** Describe how this condition typically or uniquely affects an individual's life, including specific challenges presented by the chosen focus.
            5.  **Treatment & Management Strategies (800-3000 characters):**
                * Overview of evidence-based therapies (e.g., CBT, DBT, EMDR, psychodynamic) relevant to the specific focus.
                * Pharmacological interventions (if applicable), noting any considerations for the specific scenario.
                * Self-care strategies and the importance of social support and community resources.
                * Crucially, discuss adaptations or considerations for the specific aspect of the condition being highlighted (e.g., multidisciplinary approaches for complex cases, tailored interventions for atypical presentations).
            6.  **Patient Vignette or Quote (150-400 characters):** A short, anonymized, realistic quote or brief scenario illustrating a key challenge or insight, specifically related to the discussed presentation (typical or edge case). Make it unique for this entry.
            7.  **Conclusion & Encouragement (200-500 characters):** Summarize key takeaways, emphasize hope, recovery, and encourage seeking professional help and self-compassion, with a closing thought relevant to the article's specific focus.

            **Output Format:**
            Start directly with the title, then a new line, then the content. Ensure the total content length (excluding title) is between {min_content_length} and {max_content_length} characters. Maintain medical accuracy and avoid any fabricated statistics or treatments.
            {unique_prompt_seed}
            """
        else: # style == 'patient'
            base_prompt_instructions = f"""
            You are a person bravely sharing your deeply personal, first-person story about living with {condition}. Your narrative is intended to offer solidarity, understanding, and hope to others who might be struggling or to educate those who want to understand.

            **Condition Focus:** {condition}
            **Tone:** Authentic, vulnerable, reflective, hopeful, and relatable.
            **Language:** Use natural, descriptive language.
            """
            if is_edge_case:
                edge_case_instructions = f"""
                **Special Focus (Edge Case for Patient's Story):** For this specific personal story, highlight a **less typical or particularly challenging aspect** of your journey with {condition}. Choose ONE of the following to elaborate on:
                * Your experience with **misdiagnosis** or a **very long and frustrating diagnostic journey** spanning years or multiple professionals.
                * The **late-onset of symptoms** in adulthood or elderly years, and how this differed from typical adolescent/young adult onset.
                * How the condition manifested or was perceived differently due to your **age, gender identity, or cultural/ethnic background**, leading to unique challenges.
                * Living with **severe, chronic, or highly debilitating symptoms** that significantly impact daily functioning, and the unique strategies you employ to cope.
                * Navigating the healthcare system with **unusual obstacles** (e.g., extreme geographical isolation, severe financial constraints, lack of specialized care for your specific needs).
                * Coping with {condition} alongside **major, compounding life events** (e.g., severe grief, job loss, becoming a new parent, chronic physical illness) that severely exacerbated your mental health.
                * The discovery and implementation of **unconventional or highly personalized coping strategies** that eventually worked for you after many standard approaches failed.
                * A raw and honest story of **significant relapse** or a non-linear, messy recovery path with numerous setbacks, and how you found resilience.
                * **Share a truly unique and specific personal experience** that goes beyond the common narrative.
                """
            else:
                edge_case_instructions = f"""
                **Special Focus (Typical Case for Patient's Story):** For this specific story, describe a **common or typical experience** of living with {condition}, focusing on the general onset, impact, and recovery journey. Choose ONE of the following to elaborate on:
                * The **initial, recognizable symptoms** and how they commonly manifested in your daily life.
                * The **standard impact** {condition} had on your relationships, work/school, hobbies, and overall well-being.
                * Your experience with **common therapies or coping mechanisms** (e.g., CBT, medication, support groups) that are widely used and how they helped you.
                * The process of **seeking and receiving a diagnosis** and your initial reactions to it.
                * How routine self-care practices played a role in your recovery.
                * **Provide specific, relatable details within a common experience** that resonates with many others facing {condition}. Avoid highly unusual or extreme scenarios unless contrasting with a typical experience.
                """

            prompt = f"""
            {base_prompt_instructions}
            {edge_case_instructions}

            **Required Structure & Content (Strictly Adhere to Lengths and Sections):**
            1.  **Title:** A heartfelt, evocative title (10-100 characters). Should capture the essence of your journey and hint at the specific aspect (typical or edge case).
            2.  **My Background & Early Signs (200-500 characters):** Briefly introduce yourself and describe when you first started noticing symptoms or challenges related to the condition, with details specific to your chosen focus.
            3.  **The Onset and Emotional Impact (700-2000 characters):** Detail the moment or period when the condition became undeniable. Describe the initial emotional turmoil, confusion, fear, isolation, or other feelings, particularly as experienced in your specific (typical or edge case) scenario.
            4.  **Daily Struggles and Challenges (800-3000 characters):** Paint a vivid picture of the day-to-day difficulties. How did it affect your relationships, work/school, hobbies, and overall quality of life? Include specific examples of struggles, emphasizing the unique challenges of the edge case if applicable.
            5.  **Finding Help and Coping Strategies (700-2000 characters):** Describe your journey to diagnosis and seeking help. What therapies, medications, self-help techniques, or support systems did you try? Which were helpful, and which were not? Detail specific coping mechanisms, especially those adapted for complex situations or those you found particularly effective for your specific experience.
            6.  **Current Status and Lessons Learned (300-800 characters):** Where are you now in your journey? What insights have you gained? How has your perspective changed? Reflect on the specific challenges overcome and lessons learned from your unique path.
            7.  **A Message for Others (200-500 characters):** Offer encouragement, advice, or a message of hope to others who might be experiencing similar challenges. What do you wish someone had told you? Tailor this message to those facing similar edge cases or typical struggles.

            **Output Format:**
            Start directly with the title, then a new line, then the content. Ensure the total content length (excluding title) is between {min_content_length} and {max_content_length} characters. Focus on emotional authenticity and a clear narrative arc.
            {unique_prompt_seed}
            """
        try:
            response = model.generate_content(
                prompt,
                generation_config=genai.types.GenerationConfig(
                    temperature=0.85, # Slightly higher temperature for more creativity and less repetition
                    max_output_tokens=max_content_length + 500, # Allow some buffer for title + content + formatting
                    stop_sequences=[]
                )
            )

            text = response.text.strip()

            # Robust parsing for title and content
            title = ""
            content = ""
            lines = text.split('\n', 1) # Split only on the first newline

            if len(lines) > 1:
                potential_title = lines[0].strip()
                potential_content = lines[1].strip()

                # Basic check for title length and if it looks like a title
                if 10 <= len(potential_title) <= 100 and potential_title.count('\n') == 0:
                    title = potential_title
                    content = potential_content
                else: # Fallback if first line isn't a good title
                    first_period_idx = text.find('.')
                    if first_period_idx != -1 and first_period_idx < 100:
                        title = text[:first_period_idx+1].strip()
                        content = text[first_period_idx+1:].strip()
                    else: # Last resort if no clear title/content separation
                        title = text[:min(len(text), 100)].split('\n')[0].strip() # Take first 100 chars, up to first newline
                        content = text[len(title):].strip()
            else: # If no newline at all
                title = text[:min(len(text), 100)].split('.')[0].strip() + "..." if len(text) > 100 else text.strip()
                content = text[len(title):].strip()


            # Ensure title is not too long
            if len(title) > 100:
                title = title[:97] + "..."

            # Check for failed generation content indicators from the model itself
            failed_keywords = ["failed to generate", "content generation failed", "error occurred", "quota exceeded", "i cannot fulfill this request"]
            if any(kw in content.lower() for kw in failed_keywords) or any(kw in title.lower() for kw in failed_keywords):
                print(f"Warning: API generated failure message for {condition} ({style}) entry {entry_sequence_num} (Edge Case: {is_edge_case}). Retrying...")
                time.sleep(current_delay)
                continue # Retry if it's an API-generated failure message

            # Basic content length check
            if not (min_content_length <= len(content) <= max_content_length):
                print(f"Warning: Content for {condition} ({style}) entry {entry_sequence_num} (Edge Case: {is_edge_case}) not within length limits ({len(content)} chars). Retrying...")
                time.sleep(current_delay) # Small delay for retry
                continue

            return {
                'condition': condition,
                'style': style,
                'title': title,
                'content': content,
                'is_mental_health': True, # This is the crucial label for mental health data
                'is_edge_case': is_edge_case # New column to track if it's an edge case
            }

        except ResourceExhausted as e:
            # Re-raise ResourceExhausted directly to be caught by the outer try-except
            raise e
        except GoogleAPIError as e:
            if e.response.status_code == 429: # Explicitly check for 429 (rate limit)
                # Re-raise 429 errors as ResourceExhausted for consistent outer handling
                raise ResourceExhausted(f"API Rate Limit (429) hit: {e.message}")
            else:
                print(f"Google API Error for {condition} ({style}) entry {entry_sequence_num} (Attempt {attempt+1}/{max_retries}): {e}")
                time.sleep(current_delay) # Wait for other API errors too
        except Exception as e:
            print(f"Unexpected Error generating content for {condition} ({style}) entry {entry_sequence_num} (Attempt {attempt+1}/{max_retries}): {e}")
            time.sleep(current_delay)
            continue # Try again for other errors

    # If all retries fail, return None so it's not added to the dataset
    print(f"Failed to generate valid content for {condition} ({style}) entry {entry_sequence_num} (Edge Case: {is_edge_case}) after {max_retries} attempts. Skipping this entry.")
    return None

# --- Main Generation Logic ---
output_dir = '/content/drive/MyDrive/Mental Health'
output_path = os.path.join(output_dir, 'mental_health_dataset_enhanced3.csv')
os.makedirs(output_dir, exist_ok=True)

# Load existing data if the file exists, otherwise start with an empty list
if os.path.exists(output_path):
    print(f"Loading existing mental health dataset from {output_path}...")
    df_existing = pd.read_csv(output_path)
    # Filter out any "failed to generate" or too-short entries from previous runs
    df_existing_filtered = df_existing[
        (~df_existing['title'].str.contains("Failed to generate", na=False)) &
        (~df_existing['content'].str.contains("Content generation failed", na=False)) &
        (df_existing['content'].str.len() >= 1500) # Ensure existing content meets length criteria
    ].copy() # Use .copy() to avoid SettingWithCopyWarning

    mental_health_records = df_existing_filtered.to_dict('records')
    # Use a set to track unique content hashes to prevent adding duplicates
    unique_content_hashes = set(hashlib.md5(d['content'].encode('utf-8')).hexdigest() for d in mental_health_records if 'content' in d and d['content'])

    print(f"Loaded {len(mental_health_records)} valid existing entries and {len(unique_content_hashes)} unique content hashes.")
else:
    mental_health_records = []
    unique_content_hashes = set()
    print("No existing dataset found. Starting with an empty dataset.")

# Target number of *additional* entries to generate
TARGET_ADDITIONAL_ENTRIES = 700
generated_successful_count = 0 # Counter for NEW successful entries in this run

print(f"\nAiming to generate {TARGET_ADDITIONAL_ENTRIES} more *new and unique* mental health entries.")
print(f"Dataset currently has {len(mental_health_records)} valid entries.")

# Create a list of all possible generation tasks to ensure fair distribution and uniqueness
all_generation_tasks = []
# We need 700 new entries.
# 20 diseases * 2 styles * 2 edge_case_flags = 80 base combinations.
# To get 700 unique entries, we need to iterate each combination roughly 700 / 80 = ~8.75 times.
# Let's aim for 10-12 iterations per unique (disease, style, edge_case) tuple to have a buffer for failed/duplicate generations.
num_iterations_per_variant = 12 # Adjust this if you want more or fewer total entries

for style in ['doctor', 'patient']:
    for cond in diseases:
        for is_edge_case_flag in [False, True]:
            for i in range(num_iterations_per_variant):
                all_generation_tasks.append({
                    'condition': cond,
                    'style': style,
                    'is_edge_case': is_edge_case_flag,
                    'entry_sequence_num': i + 1 # Unique sequence number for prompt diversity
                })

random.shuffle(all_generation_tasks) # Shuffle to ensure varied generation order

task_idx = 0

try:
    with tqdm(total=TARGET_ADDITIONAL_ENTRIES, desc="Generating new mental health entries") as pbar:
        # Continue generating until we hit the target or run out of unique tasks to attempt
        while generated_successful_count < TARGET_ADDITIONAL_ENTRIES and task_idx < len(all_generation_tasks):
            task = all_generation_tasks[task_idx]
            entry = generate_entry(
                task['condition'],
                task['style'],
                task['entry_sequence_num'],
                task['is_edge_case']
            )

            if entry is not None: # ONLY process if generation was successful (not None due to internal retries failing)
                # Check for content uniqueness BEFORE adding
                current_content_hash = hashlib.md5(entry['content'].encode('utf-8')).hexdigest()
                if current_content_hash not in unique_content_hashes:
                    mental_health_records.append(entry)
                    unique_content_hashes.add(current_content_hash)
                    generated_successful_count += 1
                    pbar.update(1)
                else:
                    # print(f"Skipping duplicate content generated for {task['condition']} ({task['style']}) (Edge Case: {task['is_edge_case']}).")
                    pass # Don't print for every duplicate to avoid excessive output

            task_idx += 1
            # Dynamic sleep to manage API calls without hitting limits too aggressively
            # If successful count is high, reduce sleep; if low, increase sleep
            if generated_successful_count > 0 and generated_successful_count % 50 == 0:
                 time.sleep(3) # Longer pause every 50 successful entries
            else:
                 time.sleep(1.2) # General delay for each call

            # If we've exhausted all predefined tasks but still need more entries, break.
            if task_idx >= len(all_generation_tasks) and generated_successful_count < TARGET_ADDITIONAL_ENTRIES:
                print("\nWarning: Exhausted all predefined unique generation tasks but did not reach the target count.")
                print(f"Generated {generated_successful_count} new unique entries, needed {TARGET_ADDITIONAL_ENTRIES}.")
                print("Consider increasing `num_iterations_per_variant` or adding more diseases/styles to `diseases` list.")
                break

except ResourceExhausted:
    print("\n--- API QUOTA LIMIT REACHED ---")
    print("Stopping generation to save current progress. Your data will be saved.")
except Exception as e:
    print(f"\n--- An unexpected error occurred during generation: {e} ---")
    print("Stopping generation to save current progress. Your data will be saved.")
finally:
    # This block will always execute, ensuring data is saved
    df_final = pd.DataFrame(mental_health_records)
    # Ensure no actual duplicates are in the final saved CSV due to any unforeseen issues
    df_final.drop_duplicates(subset=['content'], inplace=True)
    df_final.to_csv(output_path, index=False)
    print(f"\nFinal total unique mental health entries saved to {output_path}: {len(df_final)}")

    # --- Display Final Results ---
    print("\nSample of the final mental health data:")
    print(df_final.tail())

    print("\nEntry counts by condition, style, and edge_case status (final dataset):")
    if not df_final.empty:
        # Group by all relevant columns to see the distribution
        counts = df_final.groupby(['condition', 'style', 'is_edge_case']).size().unstack(fill_value=0)
        print(counts)
        print(f"\nTotal unique mental health entries in final dataset: {len(df_final)}")
    else:
        print("No entries in the final dataset to display counts.")

Mounted at /content/drive
No existing dataset found. Starting with an empty dataset.

Aiming to generate 700 more *new and unique* mental health entries.
Dataset currently has 0 valid entries.


Generating new mental health entries: 100%|██████████| 700/700 [1:45:53<00:00,  9.08s/it]



Final total unique mental health entries saved to /content/drive/MyDrive/Mental Health/mental_health_dataset_enhanced3.csv: 700

Sample of the final mental health data:
                                             condition    style  \
695                       generalized anxiety disorder  patient   
696                        separation anxiety disorder  patient   
697                        separation anxiety disorder   doctor   
698                    panic disorder with agoraphobia  patient   
699  borderline personality disorder (emotional dys...  patient   

                                                 title  \
695                                   The Constant Hum   
696                                The Invisible Leash   
697  Separation Anxiety: When Fear Closes In on the...   
698              The World Shrunk, But So Did My Fears   
699                            Cracked, But Not Broken   

                                               content  is_mental_health  \
69